<a href="https://colab.research.google.com/github/phbez/m_s/blob/main/16_03_Metal_Surf_Segm_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Importação das bibliotecas

In [15]:
import cv2
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization, Activation, Add, Input, AveragePooling2D
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

import numpy as np
import os, sys
from scipy import ndimage
import cv2
import matplotlib.pyplot as plt
import itertools
import scipy.stats

from keras import applications, optimizers, Input
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.multiclass import unique_labels


ModuleNotFoundError: No module named 'tensorflow'

**1.1 Segment Model**

In [8]:
#####################################################################
# SAM Model Setup
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python pycocotools matplotlib onnxruntime onnx

import os
import torch
import requests
from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator

# Create a directory for the model
!mkdir -p sam_models

# Download SAM model checkpoint
def download_sam_checkpoint(model_type='vit_b'):
    checkpoint_urls = {
        'vit_h': 'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth',
        'vit_l': 'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth',
        'vit_b': 'https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth'
    }

    checkpoint_path = f"sam_models/sam_{model_type}.pth"

    if not os.path.exists(checkpoint_path):
        print(f"Downloading {model_type} checkpoint...")
        response = requests.get(checkpoint_urls[model_type])
        with open(checkpoint_path, "wb") as f:
            f.write(response.content)
        print(f"Downloaded {model_type} checkpoint to {checkpoint_path}")
    else:
        print(f"Checkpoint already exists at {checkpoint_path}")

    return checkpoint_path

# Download and load the model
model_type = 'vit_b'  # Use smaller model for faster processing
checkpoint_path = download_sam_checkpoint(model_type)
sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sam.to(device)

# Create SAM predictor and mask generator
predictor = SamPredictor(sam)
mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.85,
    stability_score_thresh=0.9,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100
)
#####################################################################

  Cloning https://github.com/facebookresearch/segment-anything.git to c:\users\paulo\appdata\local\temp\pip-req-build-ak50pl86
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git 'C:\Users\Paulo\AppData\Local\Temp\pip-req-build-ak50pl86'


Checkpoint already exists at sam_models/sam_vit_b.pth


J� existe uma subpasta ou um arquivo -p.
Erro ao processar: -p.
J� existe uma subpasta ou um arquivo sam_models.
Erro ao processar: sam_models.


**2. Loading dataset**

In [7]:
#from google.colab import drive
#drive.mount('/content/drive')
!pip install gdown pandas numpy

import os
import gdown

# Create data directories
DATA_DIR = os.path.join(os.getcwd(), 'data')
DATASET_DIR = os.path.join(DATA_DIR, 'dataset')
os.makedirs(DATASET_DIR, exist_ok=True)

# Google Drive file ID (replace with your file ID)
file_id = '1XAt3LTujyxdU4EvBGRWbElfyUAByqoXw'
output_path = os.path.join(DATASET_DIR, 'dataset.zip')


# Download from Google Drive
url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(url, output_path, quiet=False)

# If dataset is zipped, unzip it
if output_path.endswith('.zip'):
    import zipfile
    with zipfile.ZipFile(output_path, 'r') as zip_ref:
        zip_ref.extractall(DATASET_DIR)

FileURLRetrievalError: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1XAt3LTujyxdU4EvBGRWbElfyUAByqoXw

but Gdown can't. Please check connections and permissions.

In [4]:
folder='/content/drive/MyDrive/Pós-doutorado/Dataset_M_S_T_01'

3. Estruturando o dataset

In [5]:
#image resolution
image_width = 48
image_height = 48
channels = 3 #rgb


#####################################################################
train_files = []
classes_list = ['Crazing', 'Scratches', 'Inclusion']
class_to_index = {cls: i for i, cls in enumerate(classes_list)}

for cls in classes_list:
    print(os.path.join(folder, cls))
    onlyfiles = [f for f in os.listdir(os.path.join(folder, cls)) if os.path.isfile(os.path.join(folder, cls, f))]
    for _file in onlyfiles:
        train_files.append((os.path.join(folder, cls, _file), cls))

# Function to process image with SAM
def process_with_sam(image_path):
    # Read the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Generate masks for the image
    masks = mask_generator.generate(image)

    # If no masks found, return the original image resized
    if not masks:
        return cv2.resize(image, (image_width, image_height))

    # Sort masks by area (largest first)
    masks = sorted(masks, key=lambda x: x['area'], reverse=True)

    # Get the largest mask (likely the main defect)
    mask = masks[0]['segmentation']

    # Apply the mask to the original image
    masked_image = image.copy()
    masked_image[~mask] = 0  # Set background to black

    # Resize to the required dimensions
    masked_image = cv2.resize(masked_image, (image_width, image_height))

    return masked_image

# Process and load the dataset
dataset = np.zeros((len(train_files), image_height, image_width, channels), dtype=np.float32)
y_dataset = []

print("Processing images with SAM...")
for idx, (file_path, cls) in enumerate(train_files):
    if idx % 10 == 0:
        print(f"Processing image {idx+1}/{len(train_files)}")

    # Process image with SAM
    processed_image = process_with_sam(file_path)

    # Normalize and add to dataset
    dataset[idx] = processed_image / 255.0

    # Add class label
    y_dataset.append(class_to_index[cls])

# Convert labels to categorical
y_dataset = to_categorical(y_dataset, num_classes=len(classes_list))
#####################################################################


#train_files = []
#for classes in ['Crazing', 'Scratches','Inclusion']:
    #print(os.path.join(folder, classes))
    #onlyfiles = [f for f in os.listdir(os.path.join(folder, classes)) if os.path.isfile(os.path.join(folder, classes, f ))]
    #for _file in onlyfiles:
        #train_files.append(_file)

#dataset = np.ndarray(shape=(len(train_files), image_height, image_width, channels),
                    # dtype=np.float32)
#y_dataset = []

/content/drive/MyDrive/Pós-doutorado/Dataset_M_S_T_01/Crazing
/content/drive/MyDrive/Pós-doutorado/Dataset_M_S_T_01/Scratches
/content/drive/MyDrive/Pós-doutorado/Dataset_M_S_T_01/Inclusion
Processing images with SAM...
Processing image 1/828


KeyboardInterrupt: 

In [ ]:
#i = 0
#for classes in ['Crazing', 'Scratches','Inclusion']:
    #onlyfiles = [f for f in os.listdir(os.path.join(folder, classes)) if os.path.isfile(os.path.join(folder, classes, f))]
    #for _file in onlyfiles:
        #img_path = os.path.join(folder, classes, _file)
        #img = load_img(img_path, target_size=(image_height, image_width))
        #x = img_to_array(img)
        #dataset[i] = x
        #mapping = {'Crazing': 0 , 'Scratches': 1, 'Inclusion': 2}
        #y_dataset.append(mapping[classes])
        #i += 1
        #if i == 30000:
        #    print("%d images to array" % i)
        #    break

print("All images to array!")

6. Normalizando os dados

In [ ]:
#Normalização
#dataset = dataset.astype('float32')
#dataset /= 255

**3.1 Visualize some examples with SAM segmentation**

In [ ]:
# Add this after your dataset loading to visualize some examples
#####################################################################
# Visualize some examples with SAM segmentation
def visualize_sam_examples(num_examples=5):
    plt.figure(figsize=(15, 5*num_examples))

    for i in range(min(num_examples, len(train_files))):
        # Get a random sample
        idx = np.random.randint(0, len(train_files))
        file_path, cls = train_files[idx]

        # Original image
        original = cv2.imread(file_path)
        original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)

        # Process with SAM
        processed = process_with_sam(file_path)

        # Display
        plt.subplot(num_examples, 2, i*2+1)
        plt.imshow(original)
        plt.title(f"Original - Class: {cls}")
        plt.axis('off')

        plt.subplot(num_examples, 2, i*2+2)
        plt.imshow(processed)
        plt.title(f"SAM Segmented - Class: {cls}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Visualize some examples
visualize_sam_examples()
#####################################################################

In [ ]:
classes = ['Crazing', 'Scratches', 'Inclusion']

# Dicionário para armazenar o índice da primeira imagem de cada classe
first_image_index = {}

# Encontra o índice da primeira imagem de cada classe
for i, label in enumerate(y_dataset):
    if label not in first_image_index:
        first_image_index[label] = i

# Configura a grade para exibir as imagens
num_classes = len(set(y_dataset))
num_images_per_class = 1
num_cols = num_classes
num_rows = num_images_per_class

In [ ]:
# Cria uma figura com uma grade de subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(5, 15))

# Loop através de cada classe
for i in range(num_classes):
    # Obtém o índice da primeira imagem da classe
    idx = first_image_index[i]

    # Obtém a imagem e converte para RGB
    pixels = dataset[idx]

    # Exibe a imagem no subplot correspondente
    axes[i].imshow(pixels, cmap='Spectral')
    axes[i].axis('off')

    # Adiciona um título para o subplot com o rótulo
    axes[i].set_title(f'{classes[i]}')

# Exibe a figura
plt.tight_layout()
plt.show()

7. Normalizando o dataset

In [ ]:
# categorical values
n_classes = len(set(y_dataset))
print(n_classes)

y_dataset_ = to_categorical(y_dataset, n_classes)

In [ ]:
# Ajusta o tamanho do dataset para coincidir com o tamanho menor entre dataset_trimmed e y_dataset_
min_length = min(len(dataset), len(y_dataset_))
dataset_trimmed = dataset[:min_length]
y_dataset_trimmed = y_dataset_[:min_length]

# Verifica se os tamanhos agora estão iguais
assert len(dataset_trimmed) == len(y_dataset_trimmed), "Os tamanhos ainda não são consistentes!"

# Dividindo em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(dataset_trimmed, y_dataset_trimmed, test_size=0.2)

print("Train set size: {}, Test set size: {}".format(len(X_train), len(X_test)))


In [ ]:
from sklearn.utils import resample

# Criar listas vazias para armazenar as amostras balanceadas
balanced_X_train = []
balanced_y_train = []

# Determinar o número de amostras na classe majoritária
majority_samples = 5000

# Iterar sobre cada classe
for class_label in np.unique(y_train.argmax(axis=1)):
    # Filtrar amostras pertencentes a essa classe
    X_class = X_train[y_train.argmax(axis=1) == class_label]
    y_class = y_train[y_train.argmax(axis=1) == class_label]

    # Calcular o número de amostras na classe menos representada
    minority_samples = len(X_class)

    # Balancear as amostras aumentando a classe menos representada
    balanced_X_class, balanced_y_class = resample(X_class, y_class,
                                                  replace=True,
                                                  n_samples=majority_samples,
                                                  random_state=42)

    # Adicionar amostras balanceadas à lista
    balanced_X_train.extend(balanced_X_class)
    balanced_y_train.extend(balanced_y_class)

# Converter listas em arrays numpy
balanced_X_train = np.array(balanced_X_train)
balanced_y_train = np.array(balanced_y_train)

# Embaralhar amostras
shuffled_indices = np.arange(len(balanced_X_train))
np.random.shuffle(shuffled_indices)
balanced_X_train = balanced_X_train[shuffled_indices]
balanced_y_train = balanced_y_train[shuffled_indices]

# Verificar o tamanho dos conjuntos de dados balanceados
print("Tamanho do conjunto de treinamento balanceado:", len(balanced_X_train))
print("Tamanho do conjunto de teste:", len(X_test))

for class_label in np.unique(balanced_y_train.argmax(axis=1)):
    count = np.sum(balanced_y_train.argmax(axis=1) == class_label)
    print(f"Classe {class_label}: {count} amostras")

In [ ]:
# Criar o modelo
model = Sequential()

model.add(BatchNormalization(input_shape=(image_height, image_width, 3)))
model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))  # Adiciona a camada de dropout

model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))  # Adiciona a camada de dropout

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))  # Adiciona a camada de dropout
#model.add(Dense(2, activation='softmax'))  # Especifica 'softmax' como a função de ativação
model.add(Dense(3, activation='softmax'))  # Especifica 'softmax' como a função de ativação

model.summary()

4. Compilando o modelo

In [ ]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Treinando o modelo
from tensorflow.keras.callbacks import EarlyStopping

# Configurar EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15)

# Treinar o modelo
history = model.fit(balanced_X_train, balanced_y_train,validation_split= 0.2, epochs=100, callbacks=[early_stopping], batch_size=64)


In [ ]:
val_accuracy = history.history['val_accuracy']

mean_val_accuracy = np.mean(val_accuracy)

print("Valor médio de acurácia:", mean_val_accuracy)

5. Carregando o modelo

In [ ]:
import pandas as pd
from google.colab import drive


In [ ]:
# Mount Google Drive (add this line)
drive.mount('/content/drive')

In [ ]:
# START OF SAVING CODE
# Save training history
history_save = pd.DataFrame(history.history)
#history_save.to_csv('datasetNormal.csv') # old: temporarily saving
history_save.to_csv('/content/drive/MyDrive/Pós-doutorado/Dataset_M_S_T_01/The_Model_Metal_Surfaces_v01/datasetNormal.csv')  # Changed path to Google Drive

**Save model structure in JSON file**

In [ ]:
model_json = model.to_json()
#with open("metal_surface_model.json", "w") as json_file: # old: temporarily saving
with open("/content/drive/MyDrive/Pós-doutorado/Dataset_M_S_T_01/The_Model_Metal_Surfaces_v01/metal_surface_model.json", "w") as json_file:  # Changed path
    json_file.write(model_json)

**Save complete model**

In [ ]:
#model.save('modelosalvo.keras') # old: temporarily saving
model.save('/content/drive/MyDrive/Pós-doutorado/Dataset_M_S_T_01/The_Model_Metal_Surfaces_v01/modelosalvo.keras')  # Changed path
# END OF SAVING CODE

**Load model (for verification or immediate use)**

In [ ]:
from keras.models import load_model
modelo_carregado = load_model('/content/drive/MyDrive/Pós-doutorado/Dataset_M_S_T_01/The_Model_Metal_Surfaces_v01/modelosalvo.keras')

**Load history**

In [ ]:
import pandas as pd
history = pd.read_csv('/content/drive/MyDrive/Pós-doutorado/Dataset_M_S_T_01/The_Model_Metal_Surfaces_v01/datasetNormal.csv')

6. Plotando o modelo

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history['loss'])
plt.plot(history['val_loss'])


plt.ylabel('Perda')
plt.xlabel('Época')
plt.legend(['Treinamento', 'Validação'], loc = 'upper right')
plt.show()

In [ ]:
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('Acurácia')
plt.ylabel('Acurácia')
plt.xlabel('Épocas')
plt.legend(['Treinamento', 'Validação'], loc='upper left')
plt.grid(True)
plt.show()
plt.close()

7. Salvando o modelo

In [ ]:
preds = modelo_carregado.predict(X_test)


In [ ]:
#Função da Matriz de Confusão

def plot_confusion_matrix(
        cm,
        classes,
        normalize=False,
        title='Confusion matrix',
        cmap=plt.cm.Blues
    ):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    #plt.title(title)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Classe real')
    plt.xlabel('Classe predita')

In [ ]:
#Gráfico da Matriz de Confusão

y_test_ = [np.argmax(x) for x in y_test]
preds_ = [np.argmax(x) for x in preds]

cm = confusion_matrix(y_test_, preds_)
plot_confusion_matrix(cm, classes=['Crazing', 'Scratches','Inclusion'], title='Confusion matrix')
plt.show()
plt.close()

In [ ]:
#Resultado das predições

n = 4
for t in range(4):
    plt.figure(figsize=(10,10))
    for i in range(n*t, n*(t+1)):
        plt.subplot(1, n, i + 1 - n*t)
        plt.imshow(cv2.cvtColor(X_test[i], cv2.COLOR_BGR2RGB), cmap='gray')
        plt.title('Real: {}\nPredito: {}'.format(classes[np.argmax(y_test[i])], classes[np.argmax(preds[i])]))
        plt.axis('off')
    plt.show()

# **Using the model to classify images**

**1. Load Libraries**

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from google.colab import drive
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

**2. Google Drive and Path**

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Load the pre-trained model
model_path = '/content/drive/MyDrive/Pós-doutorado/Dataset_M_S_T_01/The_Model_Metal_Surfaces_v01/modelosalvo.keras' #'/content/drive/MyDrive/modelosalvo.keras'  # Update this path to match your model's location
model = load_model(model_path)

**3. Labels**

In [ ]:
# Define class labels (update these to match your model's classes)
class_labels = ['Crazing', 'Scratches','Inclusion']

**4. Preprocessing images**

In [ ]:
def preprocess_image(image_path, target_size=(48, 48)):
    """Preprocess image for model input"""
    img = Image.open(image_path).convert('RGB')
    img = img.resize(target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img, img_array

5. **Classify Image**

In [ ]:
def classify_image(image_path):
    """Classify an image using the pre-trained model"""
    # Preprocess image
    original_img, processed_img = preprocess_image(image_path)

    # Get prediction
    prediction = model.predict(processed_img)
    predicted_class_index = np.argmax(prediction[0])
    predicted_class = class_labels[predicted_class_index]
    confidence = prediction[0][predicted_class_index] * 100

    # Display results
    plt.figure(figsize=(8, 8))
    plt.imshow(original_img)
    plt.axis('off')
    plt.title(f"This image is classified as: {predicted_class.upper()}\nConfidence: {confidence:.2f}%",
              fontsize=16, pad=20)
    plt.show()

    return predicted_class, confidence


**6. Example Usage**

In [ ]:
# Example usage
def classify_image_from_drive(relative_path):
    """Classify an image from Google Drive by providing a path relative to MyDrive"""
    full_path = f'/content/drive/MyDrive/Pós-doutorado/Dataset_M_S_T_01/Crazing_test_image/{relative_path}'
    if not os.path.exists(full_path):
        print(f"Error: File not found at {full_path}")
        return

    print(f"Processing image: {relative_path}")
    predicted_class, confidence = classify_image(full_path)
    print(f"Classification result: {predicted_class}")
    print(f"Confidence: {confidence:.2f}%")


**7. Function to let user input path**

In [ ]:
# Function to let user input a path
def classify_from_user_input():
    image_path = input("Enter the path to your image relative to Google Drive root (e.g., 'metal_surface_model/In_1.bmp'): ")
    classify_image_from_drive(image_path)

# Run the interactive function
classify_from_user_input()